# Setup

In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

## Observability & Monitoring

Phoenix is an open-source observability library designed for experimentation, evaluation, and troubleshooting. It allows AI Engineers and Data Scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.
 

In [ ]:
from phoenix.trace.langchain import LangChainInstrumentor
import phoenix as px

px.close_app()
session = px.launch_app()

LangChainInstrumentor().instrument()

In [ ]:
session.view()

# Data Loading

In [ ]:
import pandas as pd
        
df = pd.read_csv('data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv')
df

In [ ]:
import pandas as pd
        
df = pd.read_csv('data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23_subset.csv')
df

# Preprocessing & Indexing

## Splitting content into paragraphs

The content is currently stored as a list of strings. We will convert this into a single string with paragraphs separated by two newlines.

## Cleaning

In [ ]:
from src.preprocessing.preprocessor import Preprocessor

preprocesser = Preprocessor(df)
df = preprocesser.preprocess()



In [ ]:
len(df)

In [ ]:
df['content'].apply(len).describe()

In [ ]:
df.head()

# Indexing

For the indexing we use the VectorStore class which bundles the embeddings and ChromaDB.

In [ ]:
from src.vectorstore import VectorStore
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en", 
    model_kwargs={"device": "cpu"}, 
    encode_kwargs={"normalize_embeddings": True}
)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=32, length_function=len)

In [ ]:
from langchain_core.documents import Document

def create_documents(df, splitter):
    docs = []
    for index, row in tqdm(df.iterrows()):
        content = row['content']

        row = row.fillna('')

        metadata = {
            "url": row['url'],
            "domain": row['domain'],
            "title": row['title'],
            "author": row['author'],
            "date": row['date']
        }
        
        for chunk in splitter.split_text(content):
            docs.append(Document(page_content=chunk, metadata=metadata))
    
    return docs

documents = create_documents(df, recursive_text_splitter)

In [ ]:
print("ChromeDB Host: ", os.getenv('CHROMADB_HOST'))
print("ChromeDB Port: ", os.getenv('CHROMADB_PORT'))

bge_vector_store = VectorStore(embedding_function=bge_embeddings,
                               collection="cleantech-subset-analysis-1")

In the next step we will add the documents to the vector store. This will take a while depending on the number of documents.

In [ ]:
## %%script false --no-raise-error

bge_vector_store.add_documents(documents, verbose=True, batch_size=128)

After adding the documents to the vector store we can now perform similarity searches.

In [ ]:
bge_vector_store.similarity_search_w_scores("The company is also aiming to reduce gas flaring?")

# Retrieval & Generation

## First Retrieval Cooking 


In [ ]:
from src.generation import get_llm_model, LLMModel

azure_model = get_llm_model(LLMModel.GPT_3_AZURE)

In [ ]:
rag_prompt = """
Answer the question to your best knowledge when looking at the following context:
{context}
                
Question: {question}
"""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | ChatPromptTemplate.from_template(rag_prompt)
        | azure_model
        | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {
        "context": bge_vector_store.get_retriever(), 
        "question": RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

In [ ]:
rag_chain_with_source.invoke("Is the company aiming to reduce gas flaring?")

## Alternative Retrieval Cooking

In [ ]:
rag_prompt = """
Answer the question to your best knowledge when looking at the following context:
{context}
                
Question: {question}
"""

In [ ]:
rag_gpt_improved_prompt = """

Critically answer the question, using the following context and your own knowledge:

Context: {context}
Question: {question}

In case that the provided context is not sufficient, explain that the knowledge is not available but that given your own knowledge you can provide an answer which you will tag as (own knowledge).

Ensure that the answer is well-structured and provides a clear and concise response.
"""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | ChatPromptTemplate.from_template(rag_gpt_improved_prompt)
        | azure_model
        | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {
        "context": bge_vector_store.get_retriever(), 
        "question": RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

In [ ]:
rag_chain_from_docs

In [ ]:
rag_chain_with_source.invoke("Is the company aiming to reduce gas flaring?")

# Evaluation

In [ ]:
df_eval = pd.read_csv('data/Cleantech Media Dataset/cleantech_rag_evaluation_data_2024-02-23.csv')
df_eval.head()

In [ ]:
df_eval = pd.read_csv('data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23_subset_eval.csv')
df_eval.head()

In [ ]:
from src.generation import get_llm_model, LLMModel
from langchain.evaluation import load_evaluator, EvaluatorType

EVAL_TYPES = {
    "accuracy": True,
    "": True,
    
}

evaluator = load_evaluator(EvaluatorType.LABELED_SCORE_STRING, llm=get_llm_model(LLMModel.GPT_3_AZURE))

for _, item in df_eval.iterrows():
    question, relevant_chunk = item['question'], item['relevant_chunk']
    
    chain_result = rag_chain_with_source.invoke(question)
    answer, context = chain_result['answer'], chain_result['context']
    
    for key, _ in EVAL_TYPES.items():        
        score = evaluator.evaluate_strings(prediction=answer, reference=context, input=question)
        print(f"{key}: {score}")
        print()

Further try evaluating the model with Phoenix's evaluation tools. Read more: https://docs.arize.com/phoenix/evaluation/running-pre-tested-evals/retrieval-rag-relevance